In [204]:
import numpy as np
import pandas as pd
import datetime as dt
from math import trunc
from psycopg2 import sql
import assets.script as sc
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from pathlib import Path

### Example `script.py` Use

In [180]:
x = [np.random.randn(10) for i in range(100)]
y = np.random.randint(1, 100, 100)

In [181]:
df = pd.DataFrame(x, columns=['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
df = df.join(pd.Series(y, name='target'))

In [182]:
X_train, X_test, y_train, y_test = sc.split_data(df, target='target')


No columns dropped.

Target values: [ 9 29 87 64 53 64 88 32 48 19 13 41 60 64 14 28 10 39  3 89 43 52 44  7
 22 25 65 80 15 62 92 90 90 23 14 49 10 51 76 39 60 84 66 54 19 99 73 89
 18 21 42 71 87 23 94 53 70 13 73 22 85 23  8 16 86 19 55  5 92 57 55 60
 15 99  5 60 41 94 94 61 25  9 38 30 43 39 57 20 51 88 86 99 52  1 30 83
 74 15 68 46] 

Column(s) remaining: Index(['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
       'nine', 'target'],
      dtype='object')

Data is unscaled.
training sample size:  80
testing sample size:  20


In [183]:
y_pred, y_prob = sc.get_predictions(GaussianNB(), X_train, y_train, X_test)

## Midterm Project

#### Goals

The goal is to `predict arrival delays` of commercial flights. Often, there isn't much airlines can do to avoid the delays, which are often costly and . It is critical for airlines to estimate flight delays as accurate as possible because the results can be applied to improvements in customer satisfaction and income of airline agencies.

#### SQL Query

In [184]:
table_name = 'flights'
limit = 10000

query = sql.SQL(
    "SELECT DISTINCT fl_date \
        FROM {table} \
        WHERE fl_date IS NOT NULL \
        ORDER BY fl_date \
        LIMIT {limit};").format(
        table = sql.Identifier(table_name),
        limit = sql.Literal(limit)
        ,
    )
    
filename = 'flights_unique_fl_dates_10k_sample.csv'

In [185]:
sc.sql_read_tables()

0             flights
1        flights_test
2    fuel_comsumption
3         pass_sample
4          passengers
5              sample
6           temptable
7          test_table
dtype: object

In [186]:
df_flights = sc.make_csv(query, filename, overwrite=False)

File exists. Returning DataFrame...


In [187]:
df_flights['fl_date']

0      2018-01-01
1      2018-01-02
2      2018-01-03
3      2018-01-04
4      2018-01-05
          ...    
725    2019-12-27
726    2019-12-28
727    2019-12-29
728    2019-12-30
729    2019-12-31
Name: fl_date, Length: 730, dtype: object

# TABLES

### Table **flights**

In [188]:
df_flights = pd.read_csv(Path('./data') / 'flights_100k_sample.csv')
df_flights['flights'].unique()

array([1.])

In [189]:
for col in df_flights.columns:
    print(col,len(df_flights[col].unique()))
    print(df_flights[col].unique())
    

fl_date 5
['2019-07-16' '2019-07-18' '2019-07-19' '2019-07-20' '2019-07-17']
mkt_unique_carrier 10
['AA' 'UA' 'F9' 'AS' 'DL' 'WN' 'NK' 'G4' 'HA' 'B6']
branded_code_share 15
['AA_CODESHARE' 'AA' 'UA_CODESHARE' 'F9' 'AS_CODESHARE' 'DL_CODESHARE'
 'WN' 'DL' 'NK' 'G4' 'AS' 'HA' 'UA' 'HA_CODESHARE' 'B6']
mkt_carrier 10
['AA' 'UA' 'F9' 'AS' 'DL' 'WN' 'NK' 'G4' 'HA' 'B6']
mkt_carrier_fl_num 6306
[4702 4703 4704 ... 4595 5844 5064]
op_unique_carrier 26
['YX' 'AA' 'OO' 'EV' 'F9' 'OH' 'PT' 'WN' 'DL' 'ZW' 'G7' 'YV' '9E' 'MQ'
 'NK' 'CP' 'G4' 'C5' 'AX' 'AS' 'QX' 'HA' 'UA' 'KS' 'EM' 'B6']
tail_num 5492
['N443YX' 'N417YX' 'N125HQ' ... 'N177DN' 'N911DL' 'N557NW']
op_carrier_fl_num 6306
[4702 4703 4704 ... 4595 5844 5064]
origin_airport_id 371
[10397 11278 12953 12451 10693 10785 12339 11298 13930 11292 14908 14771
 13851 12264 13342 10721 13303 15304 14492 12266 14107 13891 12478 14100
 10994 11057 12889 11638 14730 14588 11433 11921 13871 12441 16218 15376
 11695 12892 11042 14869 11111 13061 14570 1

- **`fl_date`**: Flight Date (yyyy-mm-dd)
- **`mkt_unique_carrier`**: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **`branded_code_share`**: Reporting Carrier Operated or Branded Code Share Partners
- **`mkt_carrier`**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
- **`mkt_carrier_fl_num`**: Flight Number
- **`op_unique_carrier`**: Unique Scheduled Operating Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users,for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **`tail_num`**: Tail Number
- **`op_carrier_fl_num`**: Flight Number
- **`origin_airport_id`**: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **`origin`**: Origin Airport
- **`origin_city_name`**: Origin Airport, City Name
- **`dest_airport_id`**: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **`dest`**: Destination Airport
- **`dest_city_name`**: Destination Airport, City Name
- **`crs_dep_time`**: CRS Departure Time (local time: hhmm)
- **`dep_time`**: Actual Departure Time (local time: hhmm)
- **`dep_delay`**: Difference in minutes between scheduled and actual departure time. Early departures show negative numbers.	
- **`taxi_out`**: Taxi Out Time, in Minutes
- **`wheels_off`**: Wheels Off Time (local time: hhmm)
- **`wheels_on`**: Wheels On Time (local time: hhmm)
- **`taxi_in`**: 	Taxi In Time, in Minutes
- **`crs_arr_time`**: CRS Arrival Time (local time: hhmm)
- **`arr_time`**: Actual Arrival Time (local time: hhmm)
- **`arr_delay`**: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- **`cancelled`**: Cancelled Flight Indicator (1=Yes)
- **`cancellation_code`**: Specifies The Reason For Cancellation
- **`diverted`**: Diverted Flight Indicator (1=Yes)
- **`dup`**: Duplicate flag marked Y if the flight is swapped based on Form-3A data
- **`crs_elapsed_time`**: CRS Elapsed Time of Flight, in Minutes
- **`actual_elapsed_time`**: Elapsed Time of Flight, in Minutes
- **`air_time`**: Flight Time, in Minutes
- **`flights`**: Number of Flights
- **`distance`**: Distance between airports (miles)
- **`carrier_delay`**: Carrier Delay, in Minutes
- **`weather_delay`**: Weather Delay, in Minutes
- **`nas_delay`**: National Air System Delay, in Minutes
- **`security_delay`**: Security Delay, in Minutes
- **`late_aircraft_delay`**: Late Aircraft Delay, in Minutes
- **`first_dep_time`**: First Gate Departure Time at Origin Airport
- **`total_add_gtime`**: Total Ground Time Away from Gate for Gate Return or Cancelled Flight
- **`longest_add_gtime`**: Longest Time Away from Gate for Gate Return or Cancelled Flight

### Table **passengers**

In [190]:
df_passengers = pd.read_csv(Path('./data') / 'passengers_100k_sample.csv')
df_passengers

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,11.0,12100.0,66.0,13.0,0.0,750.0,9.0,114.0,81.0,...,US,United States,0,35,3,2018,1,1,F,DU
1,0.0,2.0,2200.0,12.0,12.0,0.0,0.0,37.0,56.0,50.0,...,US,United States,0,35,3,2018,1,1,F,DU
2,0.0,2.0,2200.0,12.0,12.0,0.0,0.0,37.0,46.0,40.0,...,US,United States,0,35,3,2018,1,1,L,DU
3,0.0,3.0,3300.0,18.0,11.0,0.0,0.0,35.0,68.0,59.0,...,US,United States,0,35,3,2018,1,1,L,DU
4,0.0,1.0,1100.0,6.0,1.0,0.0,0.0,13.0,15.0,12.0,...,US,United States,0,35,3,2018,1,1,F,DU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,10.0,3014224.0,0.0,0.0,786515.0,0.0,4472.0,0.0,0.0,...,BE,Belgium,8,821,2,2018,12,9,G,IF
99996,0.0,4.0,1028386.0,0.0,0.0,256883.0,0.0,4920.0,0.0,0.0,...,NL,Netherlands,8,820,2,2018,12,10,G,IF
99997,0.0,8.0,2412441.0,0.0,0.0,607721.0,0.0,4920.0,0.0,0.0,...,NL,Netherlands,8,821,2,2018,12,10,G,IF
99998,0.0,2.0,599830.0,0.0,0.0,213433.0,0.0,5004.0,0.0,0.0,...,BE,Belgium,8,821,2,2018,12,11,G,IF


In [191]:
for col in df_passengers.columns:
    print(col, len(df_passengers[col].unique()))
    print(df_passengers[col].unique())

departures_scheduled 3
[ 0. 13. 12.]
departures_performed 271
[ 11.   2.   3.   1.   5.   6.  24.  15.  16.   4.   9.   8.  17.  12.
   7.  13.  26.  10.  14.  22.  25.  33.  50.  30.  21.  20.  55.  28.
  23.  41.  27.  18.  29.  36.  19.  32. 275.  70.  39.  38.  37.  67.
  63.  31. 273.  35.  66. 110. 116. 120. 134.  42.  34.   0.  95.  97.
  60. 247. 117.  53.  61. 118.  58.  88. 103.  69. 259. 210.  68.  49.
  89.  40.  45.  48.  57. 257. 212.  54. 115. 202. 112.  78. 170.  80.
  56. 105. 106. 203.  79. 171.  81.  62.  65.  90. 139. 138.  59.  43.
 107. 101. 136. 161.  51. 100.  77. 302. 133. 301.  86.  46. 196. 231.
 114. 179. 186.  44.  47.  73. 189.  52. 181. 367. 158. 180.  94. 159.
 109. 113.  76. 119.  83. 111. 121.  99.  71.  85. 108.  93.  84. 104.
  96.  87. 102. 162. 312. 143. 167.  75.  72. 125. 128. 122. 157. 204.
 154. 155. 166. 248.  82.  64. 146. 147. 307.  74. 153. 214. 215. 197.
 199. 193. 192. 347. 160. 149. 163.  98.  91. 144. 145. 142. 150. 123.
 229. 323. 320.

In [192]:
len(df_passengers['unique_carrier_name'].unique())

300

In [193]:
count = []
for carrier in df_passengers['unique_carrier'].unique():
    count.append(df_passengers[df_passengers[['unique_carrier', 'airline_id', 'unique_carrier_name']]['unique_carrier'] == carrier]['airline_id'].unique())

len(df_passengers['airline_id'].unique())

300

- **`departures_scheduled`**: Departures Scheduled
- **`departures_performed`**: Departures Performed
- **`payload`**: Available Payload (pounds)
  - **`seats`**: Available Seats
- **`passengers`**: Non-Stop Segment Passengers Transported
  - **`freight`**: Non-Stop Segment Freight Transported (pounds)
  - **`mail`**: Non-Stop Segment Mail Transported (pounds)
- **`distance`**: Distance between airports (miles)
- **`ramp_to_ramp`**: Ramp to Ramp Time (minutes)
  - **`air_time`**: Airborne Time (minutes)
- **`unique_carrier`**: Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **`airline_id`**: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
- **`unique_carrier_name`**: Unique Carrier Name. When the same name has been used by multiple carriers, a numeric suffix is used for earlier users, for example, Air Caribbean, Air Caribbean (1).
- **`region`**: Carrier's Operation Region. Carriers Report Data by Operation Region
- **`carrier`**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
- **`carrier_name`**: Carrier Name
- **`carrier_group`**: Carrier Group Code
- **`carrier_group_new`**: Carrier Group New
- **`origin_airport_id`**: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **`origin_city_market_id`**: Origin Airport, City Market ID. City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.	
- **`origin`**: Origin Airport
- **`origin_city_name`**: Origin City
- **`origin_country`**: Origin Country Code
- **`origin_country_name`**: Origin Country
- **`dest_airport_id`**: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **`dest_city_market_id`**: Destination Airport, City Market ID. City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.
- **`dest`**: Destination Airport
- **`dest_city_name`**: Destination City
- **`dest_country`**: Destination Country Code
- **`dest_country_name`**: Destination Country
- **`aircraft_group`**: Aircraft Group
- **`aircraft_type`**: Aircraft Type
- **`aircraft_config`**: Aircraft Configuration
- **`month`**: Month
- **`year`**: Year
- **`distance_group`**: Distance Intervals, every 500 Miles, for Flight Segment
- **`class`**: Service Class

### Table **fuel_comsumption**

In [194]:
df_fuel_comsumption = pd.read_csv(Path('./data') / 'fuel_comsumption_100k_sample.csv')
df_fuel_comsumption

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5100.0,0.0,5100.0,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,396216.0,140239.0,536455.0,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848.0,0.0,0.0,0.0,...,522405.0,0.0,0.0,0.0,0.0,522405.0,569497.0,0.0,569497.0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,12,20377.0,X9,X9,Omni Air International LLC,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1584314.0,4588387.0,6172701.0,2018
3031,12,20207.0,XP,XP,XTRA Airways,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018
3032,12,20378.0,YV,YV,Mesa Airlines Inc.,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018
3033,12,20452.0,YX,YX,Republic Airline,2,21048.0,0.0,0.0,0.0,...,50043.0,0.0,0.0,0.0,0.0,50043.0,50043.0,0.0,50043.0,2018


- **`month`**: Month
- **`airline_id`**: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
- **`unique_carrier`**: Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **`carrier`**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
- **`carrier_name`**: Carrier Name
- **`carrier_group_new`**: Carrier Group New
- **`sdomt_gallons`**: Total Scheduled Domestic, Fuel Consumption (Gallons)
- **`satl_gallons`**: Scheduled Service International Atlantic - Fuel Consumption (Gallons)
- **`spac_gallons`**: Scheduled Service International Pacific - Fuel Consumption (Gallons)
- **`slat_gallons`**: Scheduled Service International Latin America - Fuel Consumption (Gallons)
- **`sint_gallons`**: Scheduled Service International Subtotal - Fuel Consumption (Gallons)
- **`ts_gallons`**: Total Scheduled Service - Fuel Consumption (Gallons)
- **`tdomt_gallons`**: Total Domestic - Fuel Consumption (Gallons)
- **`tint_gallons`**: Total International - Fuel Consumption (Gallons)
- **`total_gallons`**: Grand Total - Fuel Consumption (Gallons)
- **`sdomt_cost`**: Total Scheduled Domestic, Fuel Cost (Dollars)
- **`satl_cost`**: Scheduled Service International Atlantic - Fuel Cost (Dollars)
- **`spac_cost`**: Scheduled Service International Pacific - Fuel Cost (Dollars)
- **`slat_cost`**: Scheduled Service International Latin America - Fuel Cost (Dollars)
- **`sint_cost`**: Scheduled Service International Subtotal - Fuel Cost (Dollars)
- **`ts_cost`**: Total Scheduled Service - Fuel Cost (Dollars)
- **`tdomt_cost`**: Total Domestic - Fuel Cost (Dollars)
- **`tint_cost`**: Total International - Fuel Cost (Dollars)
- **`total_cost`**: Grand Total - Fuel Cost (Dollars)
- **`year`**: year

### Table **flights_test**


This table consists of subset of columns from table flights. It represents flights from January 2020 which will be used for evaluation. Therefore, we are missing some features that we are not suppossed to know before the flight lands.

- **`fl_date`**: Flight Date (yyyy-mm-dd)
- **`mkt_unique_carrier`**: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **`branded_code_share`**: Reporting Carrier Operated or Branded Code Share Partners
- **`mkt_carrier`**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
- **`mkt_carrier_fl_num`**: Flight Number
- **`op_unique_carrier`**: Unique Scheduled Operating Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users,for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **`tail_num`**: Tail Number
- **`op_carrier_fl_num`**: Flight Number
- **`origin_airport_id`**: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **`origin`**: Origin Airport
- **`origin_city_name`**: Origin Airport, City Name
- **`dest_airport_id`**: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **`dest`**: Destination Airport
- **`dest_city_name`**: Destination Airport, City Name
- **`crs_dep_time`**: CRS Departure Time (local time: hhmm)
- **`crs_arr_time`**: CRS Arrival Time (local time: hhmm)
- **`dup`**: Duplicate flag marked Y if the flight is swapped based on Form-3A data
- **`crs_elapsed_time`**: CRS Elapsed Time of Flight, in Minutes
- **`flights`**: Number of Flights
- **`distance`**: Distance between airports (miles)

# To Predict Flight Delay

Target variable is `arr_delay`

In [276]:
df_flights_by_month = pd.DataFrame()

for i in range(0,12):
    df_flights_by_month = pd.concat([df_flights_by_month, sc.sql_search_date(table='flights', y=2019, m=i+1).sort_values(by='fl_date')])

File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...
File exists. Returning DataFrame...


In [278]:
df_flights_by_month = df_flights_by_month[
    'fl_date',
    'mkt_unique_carrier',
    'branded_code_share',
    'mkt_carrier',
    'mkt_carrier_fl_num',
    'op_unique_carrier',
    'op_carrier_fl_num',
    'origin',
    'origin_city_name',
    'dest_airport_id',
    'dest',
    'dest_city_name',
    'crs_dep_time', 
    'dep_time',
    'dep_delay',
    'taxi_out',
    'wheels_off',
    'wheels_on',
    'taxi_in',
    'crs_arr_time',
    'arr_time',
    'arr_delay',
    'cancelled',
    'cancellation_code',
    'diverted',
    'crs_elapsed_time',
    'actual_elapsed_time',
    'air_time',
    'distance']

In [284]:
df_flights_by_month = df_flights_by_month[df_flights_by_month['cancelled'] != 1]

In [286]:
df_flights_by_month.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'op_carrier_fl_num',
       'origin', 'origin_city_name', 'dest_airport_id', 'dest',
       'dest_city_name', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out',
       'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time',
       'arr_delay', 'cancelled', 'cancellation_code', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance'],
      dtype='object')

In [195]:
count = []
for dest in df_flights['dest'].unique():
    count.append(df_flights[df_flights[['dest', 'dest_airport_id']]['dest'] == dest]['dest_airport_id'].unique())

# confirm that dest_airport_id refers to only 1 destination
len(count) == len(df_flights['dest'].unique())

True

In [196]:
# same columns between df_flights and df_passengers
print(np.intersect1d(df_flights.columns, df_passengers.columns))

['air_time' 'dest' 'dest_airport_id' 'dest_city_name' 'distance' 'origin'
 'origin_airport_id' 'origin_city_name']


In [197]:
# same columns between df_passengers and df_compsumption
print(np.intersect1d(df_passengers.columns, df_fuel_comsumption.columns))

['airline_id' 'carrier' 'carrier_group_new' 'carrier_name' 'month'
 'unique_carrier' 'year']


In [287]:
# same columns between df_flights and df_comsumption
print(np.intersect1d(df_flights.columns, df_fuel_comsumption.columns))

['month' 'year']


In [288]:
df_flights['mkt_unique_carrier'].unique()

array(['AA', 'UA', 'F9', 'AS', 'DL', 'WN', 'NK', 'G4', 'HA', 'B6'],
      dtype=object)

In [291]:
np.intersect1d(df_flights['mkt_unique_carrier'].unique(), df_fuel_comsumption['carrier'].unique())

array(['AA', 'AS', 'B6', 'DL', 'F9', 'G4', 'HA', 'NK', 'UA', 'WN'],
      dtype=object)

In [209]:
# merge tables using month and year

# make month column
for i,date in enumerate(df_flights['fl_date']):
    df_flights.loc[i, 'month'] = dt.datetime.strptime(df_flights.loc[i, 'fl_date'], '%Y-%m-%d').month
    df_flights.loc[i, 'year'] = dt.datetime.strptime(df_flights.loc[i, 'fl_date'], '%Y-%m-%d').year

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
99995    2019
99996    2019
99997    2019
99998    2019
99999    2019
Name: year, Length: 100000, dtype: int64

In [213]:
df_flights['month'] = df_flights['month'].astype(int)
df_flights['year'] = df_flights['year'].astype(int)

In [214]:
df_flights

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,year
0,2019-07-16,AA,AA_CODESHARE,AA,4702,YX,N443YX,4702,10397,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
1,2019-07-16,AA,AA_CODESHARE,AA,4702,YX,N443YX,4702,11278,DCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
2,2019-07-16,AA,AA_CODESHARE,AA,4703,YX,N417YX,4703,12953,LGA,...,0.0,33.0,0.0,0.0,NaN,NaN,NaN,NaN,7,2019
3,2019-07-16,AA,AA_CODESHARE,AA,4704,YX,N125HQ,4704,11278,DCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
4,2019-07-16,AA,AA_CODESHARE,AA,4705,YX,N410YX,4705,12451,JAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-07-20,AA,AA,AA,2136,AA,N945UW,2136,12953,LGA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
99996,2019-07-20,AA,AA,AA,2139,AA,N945UW,2139,10721,BOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
99997,2019-07-20,AA,AA,AA,2143,AA,N949UW,2143,11278,DCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019
99998,2019-07-20,AA,AA,AA,2149,AA,N945UW,2149,11278,DCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2019


In [215]:
# same columns between df_flights and df_passengers
merge_on = list(np.intersect1d(df_flights.columns, df_passengers.columns))

In [226]:
merge_on

['air_time',
 'dest',
 'dest_airport_id',
 'dest_city_name',
 'distance',
 'month',
 'origin',
 'origin_airport_id',
 'origin_city_name',
 'year']

In [256]:
df_concat = pd.merge(
    df_flights,
    df_passengers[['departures_performed', 'passengers', 'payload', 'distance_group', 'class', 'air_time', 'origin', 'dest', 'distance']],
    )

In [259]:
df_passengers

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,11.0,12100.0,66.0,13.0,0.0,750.0,9.0,114.0,81.0,...,US,United States,0,35,3,2018,1,1,F,DU
1,0.0,2.0,2200.0,12.0,12.0,0.0,0.0,37.0,56.0,50.0,...,US,United States,0,35,3,2018,1,1,F,DU
2,0.0,2.0,2200.0,12.0,12.0,0.0,0.0,37.0,46.0,40.0,...,US,United States,0,35,3,2018,1,1,L,DU
3,0.0,3.0,3300.0,18.0,11.0,0.0,0.0,35.0,68.0,59.0,...,US,United States,0,35,3,2018,1,1,L,DU
4,0.0,1.0,1100.0,6.0,1.0,0.0,0.0,13.0,15.0,12.0,...,US,United States,0,35,3,2018,1,1,F,DU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,10.0,3014224.0,0.0,0.0,786515.0,0.0,4472.0,0.0,0.0,...,BE,Belgium,8,821,2,2018,12,9,G,IF
99996,0.0,4.0,1028386.0,0.0,0.0,256883.0,0.0,4920.0,0.0,0.0,...,NL,Netherlands,8,820,2,2018,12,10,G,IF
99997,0.0,8.0,2412441.0,0.0,0.0,607721.0,0.0,4920.0,0.0,0.0,...,NL,Netherlands,8,821,2,2018,12,10,G,IF
99998,0.0,2.0,599830.0,0.0,0.0,213433.0,0.0,5004.0,0.0,0.0,...,BE,Belgium,8,821,2,2018,12,11,G,IF


In [257]:
df_concat

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,total_add_gtime,longest_add_gtime,no_name,month,year,departures_performed,passengers,payload,distance_group,class
0,2019-07-16,AA,AA,AA,2620,AA,N344PP,2620,11298,DFW,...,NaN,NaN,NaN,7,2019,1.0,0.0,63345.0,2,L
1,2019-07-16,AA,AA,AA,2630,AA,N936AN,2630,11298,DFW,...,NaN,NaN,NaN,7,2019,1.0,0.0,97800.0,1,P
2,2019-07-16,AA,AA,AA,2276,AA,N9621A,2276,11298,DFW,...,NaN,NaN,NaN,7,2019,1.0,0.0,97800.0,1,P
3,2019-07-16,UA,UA,UA,673,UA,N75435,673,11298,DFW,...,NaN,NaN,NaN,7,2019,1.0,0.0,97800.0,1,P
4,2019-07-16,AA,AA,AA,574,AA,N965TW,574,11298,DFW,...,NaN,NaN,NaN,7,2019,1.0,0.0,97800.0,1,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16168,2019-07-20,DL,DL,DL,2824,DL,N993DL,2824,10397,ATL,...,NaN,NaN,NaN,7,2019,1.0,0.0,49760.0,1,L
16169,2019-07-20,DL,DL,DL,2824,DL,N993DL,2824,10397,ATL,...,NaN,NaN,NaN,7,2019,1.0,126.0,34600.0,1,L
16170,2019-07-20,DL,DL,DL,2824,DL,N993DL,2824,10397,ATL,...,NaN,NaN,NaN,7,2019,1.0,148.0,78430.0,1,L
16171,2019-07-20,DL,DL,DL,2845,DL,N959DL,2845,14524,RIC,...,NaN,NaN,NaN,7,2019,1.0,140.0,38470.0,1,F


### `flights`

As a matter of timing, dates are important. Also gather different times during the day.
- **`fl_date`**: Flight Date (yyyy-mm-dd). Analyse monthly changes. HOT-ENCODE months, holidays.
- **`unique_carrier`**: Unique Marketing Carrier Code. <-- analise for trends first.
- **`branded_code_share`**: Reporting Carrier Operated or Branded Code Share Partners. HOT-ENCODE binary, code-shared vs not.
- **`carrier_fl_num`**: Flight Number, monthly count may reveal flight density.
- **`origin`**: Origin Airport. Less than dest. Check for null values.
- **`dest`**: Destination Airport
- **`air_time`**: Airborne Time (minutes)
- **`arr_time`**: Actual Arrival Time (local time: hhmm). Might reveal docking delays due to time of day.
- **`dep_time`**: Actual Departure Time (local time: hhmm). Might reveal passenger delays due to time of day.
- **`distance`**: Distance between airports (miles)
- **`taxi_out`**: from wheels down to gate, in Minutes
- **`taxi_in`**: from gate to wheels off, in Minutes
- **`diverted`**: Diverted Flight Indicator (1=Yes)

### `passengers`

- **`departures_performed`**: Departures Performed
- **`passengers`**: Non-Stop Segment Passengers Transported
- **`payload`**: Available Payload (pounds)
- **`distance_group`**: Distance Intervals, every 500 Miles, for Flight Segment
- **`class`**: Service Class

### ```fuel_consumption```

### `flights_test`